In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import akshare as ak

数据获取

In [142]:
symbol = "sc2309C600"
positype = "买"
r = 0.05
t = 0

In [143]:
option_df = ak.option_commodity_hist_sina(symbol=symbol)
option_df['date'] = pd.to_datetime(option_df['date'])
option_df

,date,open,high,low,close,volume
0,2023-06-13,4.95,4.95,4.00,4.50,81
1,2023-06-14,5.25,6.30,4.75,6.15,150
2,2023-06-15,6.80,6.80,4.50,4.80,85
3,2023-06-16,5.50,6.70,5.40,6.60,86
4,2023-06-19,7.05,7.25,5.75,6.30,103
5,2023-06-20,7.70,7.70,6.55,6.95,114
6,2023-06-21,7.10,11.30,6.35,10.25,346
7,2023-06-26,9.00,9.05,7.10,8.20,61
8,2023-06-27,7.85,8.85,7.45,7.80,87
9,2023-06-28,7.00,7.50,5.40,5.85,130


In [144]:
futures_df = ak.futures_zh_daily_sina(symbol="sc2309")
futures_df['date'] = pd.to_datetime(futures_df['date'])
futures_df

,date,open,high,low,close,volume,hold,settle
0,2020-09-08,430.9,430.9,412.3,430.1,4,2,425.8
1,2020-09-09,396.4,424.7,396.4,424.7,7,3,411.1
2,2020-09-10,403.1,414.4,403.1,413.0,4,5,410.1
3,2020-09-11,417.4,426.3,417.4,425.1,13,9,423.0
4,2020-09-14,410.2,425.7,410.2,416.2,4,7,415.5
...,...,...,...,...,...,...,...,...
260,2023-08-08,625.8,628.5,621.1,621.4,121615,34352,625.1
261,2023-08-09,614.6,633.8,614.6,631.3,155055,36001,624.2
262,2023-08-10,636.6,642.8,633.2,640.2,151652,37525,637.7
263,2023-08-11,637.8,642.6,635.0,642.2,140154,35661,639.0


数据处理

In [145]:
futures_df['return'] = np.log((futures_df['close']/futures_df['close'].shift(1)).fillna(1))
futures_df['sigma'] = futures_df['return'].rolling(window=30).std()*250**0.5

In [146]:
futures_df.rename(columns={'close':'S'}, inplace=True)

In [147]:
df = pd.concat([futures_df[['date','S','sigma']].set_index('date'), option_df.set_index('date')], axis=1,join='inner')

In [148]:
def option_info(symbol):
    option_type_index = next((i for i, ltr in enumerate(symbol[::-1]) if ltr in ['C', 'P']), None)
    if option_type_index is not None:
        option_type_index = len(symbol) - 1 - option_type_index
        option_type = symbol[option_type_index]
        strike_price = symbol[option_type_index+1:]
        return option_type, strike_price
    else:
        return None, None

In [153]:
def add_T(df, T):
    # 获取原 DataFrame 的行数
    num_rows = len(df)

    # 创建新列，并设置初始值为 0
    df['T'] = T

    # 从最后一行开始，依次往上加 1
    for i in range(num_rows - 2, -1, -1):
        df.loc[i, 'T'] = df.loc[i+1, 'T'] + 1
    
    df['T'] = df['T']/250

    return

In [150]:
df = df.reset_index(drop=False)

In [156]:
df[['optype','K']] = option_info(symbol)
df[['positype','r']] = positype, r
add_T(df, 0)

In [157]:
df

,date,S,sigma,open,high,low,close,volume,optype,K,positype,r,T
0,2023-06-13,510.8,0.342605,4.95,4.95,4.00,4.50,81,C,600,买,0.05,0.168
1,2023-06-14,527.0,0.354566,5.25,6.30,4.75,6.15,150,C,600,买,0.05,0.164
2,2023-06-15,518.7,0.307552,6.80,6.80,4.50,4.80,85,C,600,买,0.05,0.160
3,2023-06-16,530.9,0.313804,5.50,6.70,5.40,6.60,86,C,600,买,0.05,0.156
4,2023-06-19,529.8,0.298451,7.05,7.25,5.75,6.30,103,C,600,买,0.05,0.152
5,2023-06-20,534.9,0.299495,7.70,7.70,6.55,6.95,114,C,600,买,0.05,0.148
6,2023-06-21,549.5,0.308553,7.10,11.30,6.35,10.25,346,C,600,买,0.05,0.144
7,2023-06-26,542.0,0.310835,9.00,9.05,7.10,8.20,61,C,600,买,0.05,0.140
8,2023-06-27,542.8,0.294358,7.85,8.85,7.45,7.80,87,C,600,买,0.05,0.136
9,2023-06-28,533.1,0.300257,7.00,7.50,5.40,5.85,130,C,600,买,0.05,0.132


计算delta

In [11]:
def calculate_delta(row):
    S = float(row['S'])
    K = float(row['K'])
    r = float(row['r'])
    sigma = float(row['sigma'])
    T = float(row['T'])
    optype = row['optype']
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    
    if optype=='C':
        if positype=='买':
            delta = norm.cdf(d1)
        else:
            delta = -norm.cdf(d1)
    else:
        if positype=='买':
            delta = norm.cdf(d1)-1
        else:
            delta = 1-norm.cdf(d1)
    
    return delta

计算gamma

In [12]:
def calculate_gamma(row):
    S = float(row['S'])
    K = float(row['K'])
    r = float(row['r'])
    sigma = float(row['sigma'])
    T = float(row['T'])
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    if positype=='买':
        gamma = np.exp(-pow(d1,2)/2) / (S * sigma * np.sqrt(2*np.pi*T))
    else:
        gamma = -np.exp(-pow(d1,2)/2) / (S * sigma * np.sqrt(2*np.pi*T))
    return gamma

计算theta

In [13]:
def calculate_theta(row):
    S = float(row['S'])
    K = float(row['K'])
    r = float(row['r'])
    sigma = float(row['sigma'])
    T = float(row['T'])
    optype = row['optype']
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    theta_call = -(S*sigma*np.exp(-pow(d1,2)/2)) / (2*np.sqrt(2*np.pi*T)) - r*K*np.exp(-r*T)*norm.cdf(d2)
    
    if optype == 'C':
        if positype=='买':
            theta = theta_call
        else:
            theta = -theta_call
    else:
        if positype=='买':
            theta = theta_call + r*K*np.exp(-r*T)
        else:
            theta = -(theta_call + r*K*np.exp(-r*T))
        
    return theta

构造数据集

计算

In [14]:
df['delta'] = df.apply(calculate_delta, axis=1)

C:\Users\32872\AppData\Local\Temp\ipykernel_30556\508123895.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))


In [15]:
df['gamma'] = df.apply(calculate_gamma, axis=1)

C:\Users\32872\AppData\Local\Temp\ipykernel_30556\3851871202.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
C:\Users\32872\AppData\Local\Temp\ipykernel_30556\3851871202.py:11: RuntimeWarning: invalid value encountered in scalar divide
  gamma = np.exp(-pow(d1,2)/2) / (S * sigma * np.sqrt(2*np.pi*T))


In [16]:
df['theta'] = df.apply(calculate_theta, axis=1)

C:\Users\32872\AppData\Local\Temp\ipykernel_30556\3881850554.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
C:\Users\32872\AppData\Local\Temp\ipykernel_30556\3881850554.py:12: RuntimeWarning: invalid value encountered in scalar divide
  theta_call = -(S*sigma*np.exp(-pow(d1,2)/2)) / (2*np.sqrt(2*np.pi*T)) - r*K*np.exp(-r*T)*norm.cdf(d2)


In [17]:
df

,date,S,sigma,open,high,low,close,volume,optype,K,positype,r,T,delta,gamma,theta
0,2023-06-13,510.8,0.342605,4.95,4.95,4.00,4.50,81,C,600,买,0.05,0.168,0.154783,3.318950e-03,-54.503812
1,2023-06-14,527.0,0.354566,5.25,6.30,4.75,6.15,150,C,600,买,0.05,0.164,0.219295,3.905695e-03,-73.518397
2,2023-06-15,518.7,0.307552,6.80,6.80,4.50,4.80,85,C,600,买,0.05,0.160,0.145248,3.575962e-03,-49.043348
3,2023-06-16,530.9,0.313804,5.50,6.70,5.40,6.60,86,C,600,买,0.05,0.156,0.194263,4.180385e-03,-62.836004
4,2023-06-19,529.8,0.298451,7.05,7.25,5.75,6.30,103,C,600,买,0.05,0.152,0.172104,4.137361e-03,-56.009662
5,2023-06-20,534.9,0.299495,7.70,7.70,6.55,6.95,114,C,600,买,0.05,0.148,0.190795,4.414437e-03,-61.441654
6,2023-06-21,549.5,0.308553,7.10,11.30,6.35,10.25,346,C,600,买,0.05,0.144,0.264065,5.081708e-03,-79.810731
7,2023-06-26,542.0,0.310835,9.00,9.05,7.10,8.20,61,C,600,买,0.05,0.140,0.224889,4.756430e-03,-73.207990
8,2023-06-27,542.8,0.294358,7.85,8.85,7.45,7.80,87,C,600,买,0.05,0.136,0.210116,4.892723e-03,-67.822535
9,2023-06-28,533.1,0.300257,7.00,7.50,5.40,5.85,130,C,600,买,0.05,0.132,0.166356,4.291109e-03,-59.161458
